# Validation of TRI Modeling

gl
<br>
09.04.20

- only three sensors in reasonable proximity for 90-99 and all lead
- want to check the 2000-2018 for potential matches

In [1]:
#Libraries
import pandas as pd
from math import radians, cos, sin, asin, sqrt

In [2]:
def haversine(lon1, lat1, lon2, lat2):

    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

## Pulling the sensor data

In [3]:
#Load in TRI data from 1990 - 2018 to look for relevant sensors (run with makefile commands in processed/data_origin.txt)
TRI_base_process_90_18_nopubchem_df = pd.read_csv('/home/boogie2/Hanson_Lab/TRI_STILT/data/processed/TRI_base_process_90_18_nopubchem.csv')
TRI_base_process_90_18_nopubchem_df = TRI_base_process_90_18_nopubchem_df.drop(columns = ['Unnamed: 0'])

#While there may be duplicates in the data, we don't need them for this analysis
TRI_base_process_90_18_nopubchem_df = TRI_base_process_90_18_nopubchem_df.drop_duplicates()

#Load in EPA monitors data
EPA_mon = pd.read_csv('/home/boogie2/Hanson_Lab/TRI_STILT/data/raw/TRIChemicals_Monitors.csv')

#Interested in the monitors from 1990 to 2018
valid_monitors = EPA_mon[(EPA_mon['first_year']>=1990)]

In [4]:
print('Total TRI releases 1990-2018: {0}'.format(TRI_base_process_90_18_nopubchem_df.shape[0]))
print('\nTotal number of EPA monitors recording after 1990: {0}'.format(valid_monitors.shape[0]))
print('\nEPA tracked chemicals: ')
print(*valid_monitors['chemicalname'].drop_duplicates().values, sep = ", ")
print('\nUnique EPA Sensor Locations: {}'.format(valid_monitors.drop_duplicates(subset= ['latitude','longitude']).shape[0]))

Total TRI releases 1990-2018: 2231

Total number of EPA monitors recording after 1990: 88

EPA tracked chemicals: 
ETHYLBENZENE, STYRENE, 1,2-DIBROMOETHANE, 1,3-BUTADIENE, 1,2-DICHLOROETHANE, METHYL ISOBUTYL KETONE, TETRACHLOROETHYLENE, FORMALDEHYDE, CHLOROFORM, BENZENE, LEAD, NICKEL, CADMIUM, COBALT, DICHLOROMETHANE, ETHYLENE OXIDE, TRICHLOROETHYLENE, NAPHTHALENE, CUMENE

Unique EPA Sensor Locations: 12


In [19]:
#First need to sort the EPA sensors by ID to see which chemicals are at which facilities
valid_monitors['casnumber'] = valid_monitors['casnumber'].str.replace('-','')
epa_sensors_locs = valid_monitors.groupby(['latitude','longitude'])['casnumber'].apply(list)
epa_sensors_locs = epa_sensors_locs.reset_index()

#Gather the CAS numbers for the unique chemicals (EPA sensors)
unique_cas = epa_sensors_locs['casnumber'].to_list()
unique_cas = [item for sublist in unique_cas for item in sublist]
unique_cas =list(dict.fromkeys(unique_cas))

In [20]:
#First calculate the TRI emitters which are closest to the origin source AND have chemicals within the list of TRI emitters 
a= []

for idx in range(epa_sensors_locs.shape[0]):
    locs = TRI_base_process_90_18_nopubchem_df
    temp =epa_sensors_locs.iloc[idx] # This is EPA monitor

    #Should be using Haversin because of the rounded nature of the earth
    locs['haversine_distance_km']=locs.apply(lambda row : haversine(row['LONGITUDE'],row['LATITUDE'],temp['longitude'],temp['latitude']), axis = 1)

    #In order to add multiple entries per each - I think I will just change the iloc here to a boolean based upon distance
    matches = locs[locs['CAS#/COMPOUNDID'].isin(epa_sensors_locs['casnumber'].iloc[idx])]
    matches = matches[matches.haversine_distance_km<50]
    a.append(matches)

nearest_ls = pd.concat([epa_sensors_locs.reset_index(),pd.DataFrame(a).reset_index()],axis=1)
nearest_ls = nearest_ls.drop(columns=['index'])

#Remove any sensors which have no sensors nearby
nearest_ls = nearest_ls[nearest_ls[0].apply(lambda x: x.empty)==False]

In [21]:
#Converting the list of dataframes into one large dataframe
temp_list = []
for rows in range(nearest_ls.shape[0]):
    temp_df = nearest_ls[0].iloc[rows]
    temp_df['EPA_lat'] = nearest_ls['latitude'].iloc[rows]
    temp_df['EPA_long'] = nearest_ls['longitude'].iloc[rows]
    temp_df['casnumber'] = str(nearest_ls['casnumber'].iloc[rows])
    temp_list.append(temp_df)

EPA_TRI_merge_by_nearest_sensor = pd.concat(temp_list)
EPA_TRI_merge_by_nearest_sensor = EPA_TRI_merge_by_nearest_sensor.dropna(subset=['Group'])

In [22]:
#Cleaning up the dataframes a bit 
EPA_TRI_merge_by_nearest_sensor = EPA_TRI_merge_by_nearest_sensor[['FRSID',
                                                                    'YEAR',
                                                                    'TRIFD',
                                                                    'CAS#/COMPOUNDID',
                                                                    'CHEMICAL',
                                                                    'LATITUDE',
                                                                    'LONGITUDE',
                                                                    'EPA_lat',
                                                                    'EPA_long',
                                                                    'casnumber',
                                                                    'haversine_distance_km']]


# Expanding the output so each EPA lat/long ~ sensor ~ showcases the nearest TRI release with distance and years produced
EPA_TRI_merge_by_nearest_sensor_loc_agg = EPA_TRI_merge_by_nearest_sensor.groupby(['EPA_lat','EPA_long','LATITUDE','LONGITUDE','CHEMICAL','TRIFD','haversine_distance_km'])['YEAR'].apply(list).reset_index()
EPA_TRI_merge_by_nearest_sensor_loc_agg = pd.DataFrame(EPA_TRI_merge_by_nearest_sensor_loc_agg)
EPA_TRI_merge_by_nearest_sensor_loc_agg

,EPA_lat,EPA_long,LATITUDE,LONGITUDE,CHEMICAL,TRIFD,haversine_distance_km,YEAR
0,37.198299,-113.1506,37.037627,-113.544195,LEAD,84770STKRP1843E,39.205851,"[2012, 2013, 2014, 2015, 2016, 2017, 2018]"
1,37.198299,-113.1506,37.043001,-113.532888,LEAD,8479WSNRCC1825E,38.040146,[2018]
2,37.198299,-113.1506,37.120370,-113.556790,NICKEL,84770STGRG1301E,37.023942,"[2011, 2012, 2013, 2014, 2015]"
3,37.198299,-113.1506,37.169211,-113.423103,LEAD,8473WSNRCC155NR,24.356513,[2018]
4,37.459080,-113.2251,37.120370,-113.556790,NICKEL,84770STGRG1301E,47.743947,"[2011, 2012, 2013, 2014, 2015]"
...,...,...,...,...,...,...,...,...
593,41.842648,-111.8522,41.763580,-111.860270,LEAD,84321NVRNC1073W,8.817370,"[2010, 2011, 2012, 2013, 2014, 2015, 2016, 201..."
594,41.842648,-111.8522,41.771330,-111.848860,LEAD,8432WGNVRC2151N,7.935030,[2018]
595,41.842648,-111.8522,41.882500,-112.196400,CADMIUM,84330NCRST7285W,28.846324,"[1990, 1991]"
596,41.842648,-111.8522,41.882500,-112.196400,LEAD,84330NCRST7285W,28.846324,"[1990, 1991, 1992, 1993]"


In [23]:
#Saving the Data: 
EPA_TRI_merge_by_nearest_sensor_loc_agg.to_csv('/home/boogie2/Hanson_Lab/TRI_STILT/data/validation/EPA_validation_100.csv')